In [ ]:
# Imports
import pandas as pd
import plotly.express as px 
import ipaddress
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# read connection traffic data
df = pd.read_csv(r'/Users/Matteo-PC/Desktop/Assignment2_new/LucaBins/Idle/luca_idle.csv', encoding = 'unicode_escape')
df = df.dropna()
display(df)


In [ ]:
# Number of packets per destination IP

destinations_counts = df['Destination'].value_counts()
fig = px.bar(destinations_counts, x=destinations_counts.index, y=destinations_counts.values, width=800, height=400,
             title="Number of packets per destination",
             labels=dict(index="Destination", y="Packets count"))
fig.show()


In [ ]:
# Number of packets per bin

# A list that contains numbers of packets that were transmitted during every bin
bins = []

# Number of seconds per bin (15 min)
bin_size = 54000

# Filtered data frame for Time column
df_no_time = df[['Time']]

start_time = df_no_time['Time'].iloc[0]
prev_time = start_time
end_of_bin = start_time + bin_size
bins.append(0)
current_bin = 0

for time in df_no_time['Time'].values:
    if prev_time < time < end_of_bin or time == start_time:
        bins[current_bin] += 1
    else:
        bins.append(1)
        current_bin += 1

        start_time = time
        end_of_bin = start_time + bin_size

    prev_time = time

fig = px.bar(bins, width=800, height=400,
             title="Number of packets per bin",
             labels=dict(index="Bins", y="Number of Packets"))
fig.show()

In [ ]:
# Portion of encrypted traffic

encryption_protocols = ["TLSv1.2", "TLSv1.3", "SSL", "QUIC"]
encrypted_count = len(df[df['Protocol'].isin(encryption_protocols)])
fig = px.pie(values=[encrypted_count, len(df) - encrypted_count],
             names=["Encrypted Traffic", "Unencrypted Traffic"])
fig.show()

In [ ]:
 # check if string is a valid ip address or not
def is_ip(string):
        try:
            ipaddress.ip_address(string)
            return True
        except ValueError:
            return False

In [ ]:
def filter_fn(row):
    if (is_ip(row['Destination'])) and (ipaddress.ip_address(row['Destination']).is_private):
        return True
    else:
        return False

m = df.apply(filter_fn, axis=1)

private_request_df = df[m]
public_request_df = df[~m]

In [ ]:
test = public_request_df.tail(100)
display(test)

In [ ]:
import ipaddress
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#ipList = public_request_df['Destination'].values
ipList = test['Destination'].values


lookup_table = [
    ('Google','https://raw.githubusercontent.com/lord-alfred/ipranges/main/google/ipv4.txt'),
    ('Google','https://raw.githubusercontent.com/lord-alfred/ipranges/main/google/ipv6.txt'),
    ('Bing','https://raw.githubusercontent.com/lord-alfred/ipranges/main/bing/ipv4.txt'),
    ('Amazon','https://raw.githubusercontent.com/lord-alfred/ipranges/main/amazon/ipv4.txt'),
    ('Amazon','https://raw.githubusercontent.com/lord-alfred/ipranges/main/amazon/ipv6.txt'),
    ('Microsoft','https://raw.githubusercontent.com/lord-alfred/ipranges/main/microsoft/ipv4.txt'),
    ('Microsoft','https://raw.githubusercontent.com/lord-alfred/ipranges/main/microsoft/ipv6.txt'),
    ('Oracle','https://raw.githubusercontent.com/lord-alfred/ipranges/main/oracle/ipv4.txt'),
    ('DigitalOcean','https://raw.githubusercontent.com/lord-alfred/ipranges/main/digitalocean/ipv4.txt'),
    ('DigitalOcean','https://raw.githubusercontent.com/lord-alfred/ipranges/main/digitalocean/ipv6.txt')
]
"""
lookup_table = [
    ('Google','https://raw.githubusercontent.com/lord-alfred/ipranges/main/google/ipv4.txt')
]
"""
for index, tuple in enumerate(lookup_table):

    target_url = tuple[1]
    cloud_name = tuple[0]

    for elem in ipList:
        for line in urlopen(target_url):
            value = line.decode('utf-8').strip()
            if ipaddress.ip_address(elem) in ipaddress.ip_network(value):
                #public_request_df['Cloud_Service'] = cloud_name
                test['Cloud_Service'] = cloud_name


Active / Idle bins basic metrics

In [ ]:
# IDLE: number of packets
idle = pd.read_csv (r'/Users/Matteo-PC/Desktop/Assignment2_new/LucaBins/Idle/luca_idle.csv', encoding = 'unicode_escape')
idle = idle.dropna()
print('# packets during idle: ', idle.size)
idle_avg = idle.size/30
print('Avg packet rate per hour: ', idle_avg)
idle_sum = idle['Length'].sum()
idle_sum = idle_sum/1000000
print('Total amount of idle data exchanged (MB): ', idle_sum)
print('Idle data exchanged per hour (MB/hour): ', idle_sum/30)
# idle hours (luca): 122 bins = approx 30 hours

# ACTIVE: number of packets
active = pd.read_csv (r'/Users/Matteo-PC/Desktop/Assignment2_new/LucaBins/Active/luca_active.csv', encoding = 'unicode_escape')
active = active.dropna()
print('# packets during active: ', active.size)
active_avg = active.size/20
print('Avg packet rate per hour: ', active_avg)
active_sum = active['Length'].sum()
active_sum = active_sum/1000000
print('Total amount of active data exchanged (MB): ', active_sum)
print('Active data exchanged per hour (MB/hour): ', active_sum/20)
# active hours (luca): 78 bins = approx 20 hours


Group by device type

In [ ]:
lookup_table = pd.read_excel (r'/Users/Matteo-PC/Desktop/Assignment2_new/device_type.xlsx')
display(lookup_table)
lookup_dict = dict(zip(lookup_table["ip_address"],lookup_table["device_type"]))
    
ipList = private_request_df['Destination'].values

for elem in ipList:
    for index_ips in lookup_dict:
        if(elem == index_ips):
            private_request_df['Device_Type'] = lookup_dict[index_ips]
                

In [ ]:
fake_graph_table = pd.read_excel (r'/Users/Matteo-PC/Desktop/Assignment2_new/device_type_distribution.xlsx')
display(fake_graph_table)
fig = px.pie(fake_graph_table, names = 'device_type', values = 'percentage', title = 'Number of packets per device type')
fig.show()

In [ ]:
fig = px.pie(private_request_df, names = 'Device_Type', title = 'Number of packets per device type')
fig.show()

Geolocation of IP addresses

In [ ]:
ipList = public_request_df['Destination'].values
ipList = list(dict.fromkeys(ipList))       
textfile = open("/Users/Matteo-PC/Desktop/Assignment2_new/ip_geolocation.txt", "w")
for element in ipList:
    try:
        if(isinstance(ipaddress.ip_address(element),ipaddress.IPv4Address)):
            textfile.write(element + "\n")
    except ValueError:
        pass #do nothing!
textfile.close()
# then use the .txt file in 
# https://ipinfo.io/tools/map


Active / Idle bins basic metrics

In [ ]:
# IDLE: number of packets
idle = pd.read_csv (r'/Users/Matteo-PC/Desktop/Assignment2_new/LucaBins/Idle/luca_idle.csv', encoding = 'unicode_escape')
idle = idle.dropna()
print('# packets during idle: ', idle.size)
idle_avg = idle.size/30
print('Avg packet rate per hour: ', idle_avg)
idle_sum = idle['Length'].sum()
idle_sum = idle_sum/1000000
print('Total amount of idle data exchanged (MB): ', idle_sum)
print('Idle data exchanged per hour (MB/hour): ', idle_sum/30)
# idle hours (luca): 122 bins = approx 30 hours

# ACTIVE: number of packets
active = pd.read_csv (r'/Users/Matteo-PC/Desktop/Assignment2_new/LucaBins/Active/luca_active.csv', encoding = 'unicode_escape')
active = active.dropna()
print('# packets during active: ', active.size)
active_avg = active.size/20
print('Avg packet rate per hour: ', active_avg)
active_sum = active['Length'].sum()
active_sum = active_sum/1000000
print('Total amount of active data exchanged (MB): ', active_sum)
print('Active data exchanged per hour (MB/hour): ', active_sum/20)
# active hours (luca): 78 bins = approx 20 hours


Group by device type

In [ ]:
lookup_table = pd.read_excel (r'/Users/Matteo-PC/Desktop/Assignment2_new/device_type.xlsx')
display(lookup_table)
lookup_dict = dict(zip(lookup_table["ip_address"],lookup_table["device_type"]))
    
ipList = private_request_df['Destination'].values

for elem in ipList:
    for index_ips in lookup_dict:
        if(elem == index_ips):
            private_request_df['Device_Type'] = lookup_dict[index_ips]
                

In [ ]:
fake_graph_table = pd.read_excel (r'/Users/Matteo-PC/Desktop/Assignment2_new/device_type_distribution.xlsx')
display(fake_graph_table)
fig = px.pie(fake_graph_table, names = 'device_type', values = 'percentage', title = 'Number of packets per device type')
fig.show()

In [ ]:
fig = px.pie(private_request_df, names = 'Device_Type', title = 'Number of packets per device type')
fig.show()

Geolocation of IP addresses

In [ ]:
ipList = public_request_df['Destination'].values
ipList = list(dict.fromkeys(ipList))       
textfile = open("/Users/Matteo-PC/Desktop/Assignment2_new/LucaBins/ip_geolocation.txt", "w")
for element in ipList:
    try:
        if(isinstance(ipaddress.ip_address(element),ipaddress.IPv4Address)):
            textfile.write(element + "\n")
    except ValueError:
        pass #do nothing!
textfile.close()
# then use the .txt file in 
# https://ipinfo.io/tools/map


Match public DNS Service IP Address to rows in the dataframe

In [ ]:
df.loc[df['Source'].str.contains("8.8.8.8"),'DNS_Service'] = 'Google'
df.loc[df['Source'].str.contains("8.8.4.4"),'DNS_Service'] = 'Google'
df.loc[df['Source'].str.contains("9.9.9.9"),'DNS_Service'] = 'Quad9'
df.loc[df['Source'].str.contains("149.112.112.112"),'DNS_Service'] = 'Quad9'
df.loc[df['Source'].str.contains("208.67.222.222"),'DNS_Service'] = 'OpenDNS'
df.loc[df['Source'].str.contains("208.67.220.220"),'DNS_Service'] = 'OpenDNS'
df.loc[df['Source'].str.contains("1.1.1.1"),'DNS_Service'] = 'Cloudflare'
df.loc[df['Source'].str.contains("1.0.0.1"),'DNS_Service'] = 'Cloudflare'
df.loc[df['Source'].str.contains("185.228.168.9"),'DNS_Service'] = 'CleanBrowsing'
df.loc[df['Source'].str.contains("185.228.169.9"),'DNS_Service'] = 'CleanBrowsing'
df.loc[df['Source'].str.contains("76.76.19.19"),'DNS_Service'] = 'Alternate DNS'
df.loc[df['Source'].str.contains("76.223.122.150"),'DNS_Service'] = 'Alternate DNS'
df.loc[df['Source'].str.contains("94.140.14.14"),'DNS_Service'] = 'AdGuard DNS'
df.loc[df['Source'].str.contains("94.140.15.15"),'DNS_Service'] = 'AdGuard DNS'

#google = df.loc[df['Source'].str.contains("8.8.8.8", case=False)]
display(df)

IP Version Analysis

In [ ]:
df.loc[df['Source'].str.contains("."),'IP_Version'] = 'ipv4'
df.loc[df['Source'].str.contains(":"),'IP_Version'] = 'ipv6'


In [ ]:
fig = px.histogram(df, x=df['IP_Version'], marginal = "box",width=800, height=400)
fig.show()

In [ ]:
#fig = px.scatter(df, x='Time', y='Source', color='Protocol')
#fig.show()

Protocol Analysis

In [ ]:
fig = px.box(df, x='Length', color='Protocol')
fig.show()

In [ ]:
# length sum for each protocol
prtocol_length_sum_df = df.groupby(['Protocol'])['Length'].agg('sum').reset_index()
fig = px.bar(prtocol_length_sum_df, x='Protocol', y='Length')
fig.show()

In [ ]:
fig = px.histogram(df, x=df['Protocol'], marginal = "box",width=800, height=400)
fig.show()

Length Analysis

In [ ]:
fig = px.histogram(df, x=df['Length'], marginal = "box",width=800, height=400)
fig.show()

In [ ]:
fig = px.histogram(df, x=df['Length'], color=df['Protocol'])
fig.show()